In [1]:
from main import init_config
from controller import Controller
from dataset.process import TokenParser

import numpy as np

ctrl = Controller(*init_config())

2021-12-23 00:19:07.924621: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Running the DreamProlog algorithm.
There was a problem with the provided arguments. The program will run in the default setting:
--configs prolog --logdir logdir
Namespace(act='elu', action_gnn_hidden_act=64, action_gnn_hidden_val=200, action_gnn_layers=6, action_gnn_next_shape=(32, 64, 32), action_gnn_start_shape=(4, 1, 4), action_repeat=1, actor_disc=5, actor_dist='normal', actor_entropy='1e-4', actor_grad_clip=100, actor_init_std=1.0, actor_layers=4, actor_lr=0.001, actor_min_std=0.1, actor_outscale=0.0, actor_state_entropy=0.0, actor_temp=0.1, atari_grayscale=False, behavior_stop_grad=True, clip_rewards='identity', collect_dyn_sample=True, dataset_size=100000, debug=False, disag_layers=1, disag_log=True, disag_models=3, disag_offset=0, disag_target='embed', disag_units=200, discount=0.95, discount_lambda=0.8, discount_layers=2, discount_scale=1.0, dyn_cell='gru', dyn_deter=512, dyn_discrete=32, dyn_hidden=512, dyn_input_layers=5, dyn_mean_act='none', dyn_min_std=0.1, dyn_output_lay

usage: ipykernel_launcher.py [-h] --configs CONFIGS [CONFIGS ...] --logdir
                             LOGDIR
ipykernel_launcher.py: error: the following arguments are required: --configs, --logdir
2021-12-23 00:19:09.357242: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-23 00:19:09.357900: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-23 00:19:09.395446: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-23 00:19:09.396111: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemor

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t46_subset_1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t54_xboole_1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Prefill dataset (0 steps).
Loaded problem: m2n140__t6_funct_1
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t29_funct_1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Eval episode has 16 steps and return -0.200.


/home/kristsz/DreamProLog/envs/wrappers.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  value = np.array(value)


In [2]:
meta = ctrl.datasetManager._train_eps._meta
ds = []
for v in meta.values():
    ds.extend([str(s)[1:] for s in v['action_space_text']])
    
parser = TokenParser()

def pad(narr):
    size = narr.size
    return np.pad(narr, [0, 128-size])

parsed_ds = []
for dp in ds:
    parsed_ds.append(pad(np.array(parser.parse(dp), dtype = np.int32)))

In [3]:
import tensorflow as tf
BATCH_SIZE = 32
def generator():
    while True:
        x = tf.constant([parsed_ds[np.random.randint(len(parsed_ds))] for i in range(BATCH_SIZE)])
        yield x, x
sgn = tf.TensorSpec((BATCH_SIZE, 128), dtype=tf.int32)
tf_ds = tf.data.Dataset.from_generator(generator, output_signature = (sgn, sgn))

2021-12-23 00:19:19.744340: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-23 00:19:19.961684: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-23 00:19:19.962339: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-12-23 00:19:19.962366: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] 

In [4]:
from unorganized.parts import Net, Encoder

GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [5]:
class Loss(tf.keras.losses.Loss):
    def __init__(self, omega=0.01):
        super().__init__()
        self.omega = omega
        
    def call(self, y_true, y_pred):
        y = tf.nn.softmax(y_pred, axis=-1)
        y_probs = tf.gather(y, y_true, axis=-1, batch_dims=2)
        y_log_probs = tf.math.log(y_probs+0.001)
        #loss = self.omega*y_log_probs + (1-self.omega)*tf.math.cumsum(y_log_probs, axis=-1)
        #loss = y_log_probs*tf.math.cumprod(0.97*tf.ones((32,128)), axis=-1)
        mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
        mask = tf.math.cumprod(mask, axis=-1)
        loss = (mask+self.omega)*y_log_probs
        loss = -tf.math.reduce_sum(loss)/(128*32)
        return loss

In [6]:
model = Net(256, 16, 512)
#d_model = 256
#enc_embed = tf.keras.layers.Embedding(300, d_model)
#encoder = Encoder(4, d_model, 4, 512, 0.04)

In [7]:
x, x = next(iter(tf_ds))
y_true = x
mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
mask = tf.math.cumprod(mask, axis=-1)

2021-12-23 00:19:20.910756: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-23 00:19:20.929486: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3598190000 Hz


In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss=Loss(), metrics=[])


In [14]:
model.fit(tf_ds, epochs = 1)

   1832/Unknown - 105s 57ms/step - loss: 0.6950

KeyboardInterrupt: 

In [13]:

x, y_true = next(iter(tf_ds))
y_pred = model(x, False)
y_pred = tf.nn.softmax(y_pred, axis=-1)
y_probs = tf.gather(y_pred, y_true, axis=-1, batch_dims=2)
y_pred = tf.math.argmax(y_pred, axis=-1)
print(x[:4], y_pred[:4], tf.cast(100*y_probs[:4], dtype=tf.int32), sep='\n\n')

tf.Tensor(
[[  9   5  12  13   2   3  21   5  13  16   2  21   5  12  18   2   9   5
   18  16   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  9   5  12  13   2   3  76  23  13   2  76  23  12   2   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   